# Solution of k-SAT formulas via BP + reinforcement


Draw a random 3-SAT formula and try to solve it using Belief Propagation. Reinforcement is necessary to break the symmetry: BP gives a uniform measure over all SAT assignments, we are only interested in finding one

In [13]:
using BeliefPropagation, BeliefPropagation.Models
using Random, IndexedFactorGraphs

Draw a random formula with $N$ variables and $M$ clauses

In [18]:
rng = MersenneTwister(0)
n = 100
m = 50
k = 3
g = rand_regular_factor_graph(rng, n, m, k)
ψ = [KSATClause(bitrand(rng, length(neighbors(g, factor(a))))) for a in factors(g)]
bp = fast_ksat_bp(g, ψ);

Run BP

In [19]:
iters = iterate!(bp; maxiter=1000, tol=1e-14, rein=5e-2)

43

Given the beliefs $\{b_i\}_i$, our guess for the solving assignment is
$$ x_i^* = \arg\max_{x_i} b_i(x_i)$$

In [20]:
xstar = argmax.(beliefs(bp)) .- 1   # the -1 is needed because beach belief has indices (1,2), corresponding to values (0,1), respectively
xstar'

1×1000 adjoint(::Vector{Int64}) with eltype Int64:
 0  0  0  1  0  0  1  0  1  1  0  0  1  …  0  0  0  0  0  0  0  1  0  0  0  1

Check that the assignment satisfies the formula:

In [21]:
nunsat = sum(!(Bool(bp.ψ[a](xstar[i]+1 for i in neighbors(bp.g, factor(a))))) 
    for a in factors(bp.g))
println("Number of unsatisfied clauses: $nunsat")

Number of unsatisfied clauses: 0
